In [ ]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import findspark
findspark.init()

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Clustering using SparkML").getOrCreate()

24/06/10 14:14:36 WARN Utils: Your hostname, ubuntu-MS-7D15 resolves to a loopback address: 127.0.1.1; using 192.168.1.3 instead (on interface enp5s0)
24/06/10 14:14:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/10 14:14:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Clustering 1

In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/customers.csv

--2024-06-10 14:14:53--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/customers.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8909 (8.7K) [text/csv]
Saving to: ‘customers.csv.1’

customers.csv.1     100%[===================>]   8.70K  --.-KB/s    in 0s      

2024-06-10 14:14:54 (408 MB/s) - ‘customers.csv.1’ saved [8909/8909]



In [4]:
customer_data = spark.read.csv("customers.csv", header=True, inferSchema=True)
customer_data.show(n=5, truncate=False)

+----------+----+-------+-----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|
+----------+----+-------+-----------+
|12669     |9656|7561   |214        |
|7057      |9810|9568   |1762       |
|6353      |8808|7684   |2405       |
|13265     |1196|4221   |6404       |
|22615     |5410|7198   |3915       |
+----------+----+-------+-----------+
only showing top 5 rows



In [5]:
# Assemble the features into a single vector column
feature_cols = ['Fresh_Food', 'Milk', 'Grocery', 'Frozen_Food']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
customer_transformed_data = assembler.transform(customer_data)

In [6]:
number_of_clusters = 3
kmeans = KMeans(k = number_of_clusters)

#### Train/fit model

In [7]:
model = kmeans.fit(customer_transformed_data)

24/06/10 14:14:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
24/06/10 14:14:57 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


#### Predictions

In [8]:
predictions = model.transform(customer_transformed_data)

In [9]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   49|
|         2|   60|
|         0|  331|
+----------+-----+



In [10]:
spark.stop()

### Clustering 2

In [11]:
spark = SparkSession.builder.appName("Seed Clustering").getOrCreate()

In [12]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv

--2024-06-10 14:14:58--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8973 (8.8K) [text/csv]
Saving to: ‘seeds.csv.1’

seeds.csv.1         100%[===================>]   8.76K  --.-KB/s    in 0s      

2024-06-10 14:14:59 (458 MB/s) - ‘seeds.csv.1’ saved [8973/8973]



In [13]:
seed_data = spark.read.csv("seeds.csv", header=True, inferSchema=True)
seed_data.show(n=5, truncate=False)

+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+
|area |perimeter|compactness|length of kernel|width of kernel|asymmetry coefficient|length of kernel groove|
+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+
|15.26|14.84    |0.871      |5.763           |3.312          |2.221                |5.22                   |
|14.88|14.57    |0.8811     |5.554           |3.333          |1.018                |4.956                  |
|14.29|14.09    |0.905      |5.291           |3.337          |2.699                |4.825                  |
|13.84|13.94    |0.8955     |5.324           |3.379          |2.259                |4.805                  |
|16.14|14.99    |0.9034     |5.658           |3.562          |1.355                |5.175                  |
+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+
only showing top 5 

In [14]:
seed_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length of kernel: double (nullable = true)
 |-- width of kernel: double (nullable = true)
 |-- asymmetry coefficient: double (nullable = true)
 |-- length of kernel groove: double (nullable = true)



In [15]:
feature_cols = ['area', 'perimeter', 'compactness', 'length of kernel', 'width of kernel', 'asymmetry coefficient', 'length of kernel groove']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
seed_transformed_data = assembler.transform(seed_data)

In [16]:
number_of_clusters =  7
kmeans = KMeans(k = number_of_clusters)

In [17]:
model = kmeans.fit(seed_transformed_data)

#### Predictions

In [18]:
predictions = model.transform(seed_transformed_data)

In [19]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   17|
|         6|   29|
|         3|   33|
|         5|   63|
|         4|   17|
|         2|   15|
|         0|   36|
+----------+-----+



In [20]:
spark.stop()